In [2]:
import zipfile
with zipfile.ZipFile('twitter_cikm_2010.zip', 'r') as zip_ref:
    zip_ref.extractall('tweets')

In [27]:
import numpy as np

file_path = 'tweets/training_set_tweets.txt'

# Step 1: Open the file
with open(file_path, 'r') as file:
    # Step 2: Read the contents
    contents = file.read()

# Step 3: Split into lines
lines = contents.splitlines()[:1000]

tweets = np.array([])
# Step 4: Extract data from each line
for line in lines:
    fields = line.split('\t')
    if len(fields) >= 4:  # Ensure the line has at least 4 fields
        #data1 = fields[0]
        #data2 = fields[1]
        data3 = fields[2]
        #data4 = fields[3]
        tweets = np.append(tweets, data3)
        # Step 5: Process and use the extracted data
        #print(f"Data 1: {data1}")
        #print(f"Data 2: {data2}")
        print(f"Data 3: {data3}")
        #print(f"Data 4: {data4}")
        
print(tweets.shape)

Data 3: @thediscovietnam coo.  thanks. just dropped you a line.
Data 3: @thediscovietnam shit it ain't lettin me DM you back, what's your email?
Data 3: @thediscovietnam hey cody, quick question...can you dm me?
Data 3: @smokinvinyl dang.  you need anything?  I got some left over meds!
Data 3: maybe i'm late in the game on this one, but this lowender vst is making my apt rumble!
Data 3: i really hope A.I. makes the most of this second chance in philly. i'm glad he's goin home.
Data 3: @smokinvinyl danny boy! wanna check out d-nice at the afex 1 year tonight?  we could pre-game at mine and walk over.
Data 3: ...and if you have ppl that you care about, make sure to let them know. life is too short to lose friends over bullshit.  peace and love.
Data 3: ...that shit weighs heavy on me.  take responsibility for your life. I don't blame anyone for where I am in this world...
Data 3: ...including his last failed relationship.  and while I know that none of it is grounded in reality, and I ac

In [60]:
import pandas as pd
file_path = 'tweets/training_set_tweets.txt'
df = pd.read_csv(file_path, delimiter='\t', header=None, on_bad_lines='skip')

df_short = df.head(5000).dropna()  #iloc[:, 2]
df_short.columns=['a', 'b', 'Text', 'd']
print(df.shape)
print(df_short.shape)
print(df_short.columns)
print(df_short.Text)
print(df_short.isna().sum())


(2118325, 4)
(4749, 4)
Index(['a', 'b', 'Text', 'd'], dtype='object')
0       @thediscovietnam coo.  thanks. just dropped yo...
1       @thediscovietnam shit it ain't lettin me DM yo...
2       @thediscovietnam hey cody, quick question...ca...
3       @smokinvinyl dang.  you need anything?  I got ...
4       maybe i'm late in the game on this one, but th...
                              ...                        
4995           @SapphicOwl can't wait for cooler weather!
4996                       @Ingaxo dang that sounds good!
4997    @TeffyBby same. Just eating a sandwich and wat...
4998                            @TeffyBby Whats been up??
4999                            @eeayelle Tampa Bay area.
Name: Text, Length: 4749, dtype: object
a       0
b       0
Text    0
d       0
dtype: int64


In [61]:
import openai
from openai.embeddings_utils import get_embedding
model='text-embedding-ada-002'

openai.api_key = '**'
i = iter(list(range(1, 10000)))

def get_embedding(text, model="text-embedding-ada-002"):
   print(next(i))
   text = text.replace("\n", " ")
   return openai.Embedding.create(input = [text], model=model)['data'][0]['embedding']

print(df_short)

df_short['Embedding'] = df_short['Text'].apply(lambda x: get_embedding(x, model))
print(df_short.shape)
df_short.to_csv('embedded_1k_tweets.csv', index=False)


             a           b                                               Text  \
0     60730027  6320951896  @thediscovietnam coo.  thanks. just dropped yo...   
1     60730027  6320673258  @thediscovietnam shit it ain't lettin me DM yo...   
2     60730027  6319871652  @thediscovietnam hey cody, quick question...ca...   
3     60730027  6318151501  @smokinvinyl dang.  you need anything?  I got ...   
4     60730027  6317932721  maybe i'm late in the game on this one, but th...   
...        ...         ...                                                ...   
4995  21408463  4870967764         @SapphicOwl can't wait for cooler weather!   
4996  21408463  4869871268                     @Ingaxo dang that sounds good!   
4997  21408463  4869637705  @TeffyBby same. Just eating a sandwich and wat...   
4998  21408463  4869257702                          @TeffyBby Whats been up??   
4999  21408463  4868875687                          @eeayelle Tampa Bay area.   

                        d  

(96, 5)


NameError: name 'column' is not defined

In [73]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np


def get_embedding(text, model="text-embedding-ada-002"):
   text = text.replace("\n", " ")
   return openai.Embedding.create(input = [text], model=model)['data'][0]['embedding']

input_sentence = "Stay safe out there, folks! 🚗 Remember to always park in well-lit areas, lock your vehicle, and be aware of your surroundings. Let's work together to bring down carjacking incidents in our community. #PublicSafety #CrimePrevention"
input_embedding = np.array(get_embedding(input_sentence, model))




df = pd.read_csv('embedded_1k_tweets.csv')

df['Embedding'] = df['Embedding'].apply(eval).apply(np.array)

embeddings = np.array(df['Embedding'].tolist())



similarities = cosine_similarity([input_embedding], embeddings)


sorted_indices = np.argsort(similarities[0])[::-1]

print(embeddings.shape)

for i in sorted_indices[:10]:
    print(similarities[0][i])

top_10_tweets = [df['Text'][idx] for idx in sorted_indices[:10]]

for tweet in top_10_tweets:
    print(tweet)
    print("###########################")





(4749, 1536)
0.7953636629599811
0.7932163696963044
0.788139065104496
0.7867493403605215
0.7858772773065147
0.784123598332255
0.7832882921329496
0.7830378587804856
0.7819741154005471
0.780933738967966
This beautiful weekend has begun... Happy long weekend.  Safe & Fun is the goal
###########################
Walking to jeffrey's for more raw catfood. Yeah, I know how to party. Cars are scary-especially when they honk:(
###########################
@MattMajorFLP on the news 2day they said 1 camden 2 st louis 3 detroit for most dangerous cities lol... Hope we both make it back ;-)
###########################
Cars on either side of the road + kids makes driving impossible in this neighborhood  http://yfrog.com/0540106253j
###########################
6th Ave barricades are triggering a PTSD-like visions of wallet stealing and ass grabbing...shaking it off
###########################
Our sign is illuminated once more.
###########################
Happy Fall everyone!
###########################

In [71]:
print(len(df['Embedding'][0]))

1536
